

Infi-flux License

Copyright (c) 2023 Infi-flux

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE AND HYPERLINKS LINKED TO EXTERNAL SITES PROOVIDED BELOW OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE AND HYPERLINKS LINKED TO EXTERNAL SITES PROOVIDED IN THIS DIALOGUE.




# Objective

To measure the current possible market maximum potential based on recent historic data for a particular behaviour of an investment strategy for a particular instrument for bull and bear sentiment catering to intraday and long term type/style of investing using simple concept of Central limit theorem which is taking MEAN / Average of the returns over specified interval.  

# Definition

CLT

The Game of Curiosity revolves around a fundamental concept:  [Central Limit Theorem](https://youtu.be/zeJD6dqJ5lo?si=Ejvq3dBLcRlIF_ms) [ [CLT](https://www.investopedia.com/terms/c/central_limit_theorem.asp) ]. This notebook marks the inception of the Preliminary Market Analysis, the first phase of our Generic Basic Investment Architecture (GBIA). It serves as an introduction to one of the cornerstone statistical concepts applicable in analyzing Financial Markets.

The CLT, a cornerstone of statistics, is pivotal in probability theory and data analysis. It asserts that the sum of independent random variables, regardless of their distribution, tends toward a normal distribution (Gaussian distribution) with a sufficiently large sample size.

However, a word of caution: our domain involves complex systems rooted in human psychology. Thus, the applicability of CLT hinges on various factors such as market behaviors, timeframes, and dynamics like trends, news events, shocks, dividends, and changes in management or product launches. Understanding the intricacies of Financial market dynamics demands domain-specific knowledge.

In essence, the adequacy of the sample size for analysis employing CLT is contingent upon multiple variables, including stated objectives. As our data primarily comprises time series, recent data better reflects the current state of the stock market, characterized by variables like trends.

Determining the ideal sample size for analysis is not formulaic, as investment goals vary significantly among individuals. Drawing from our experience and perspective, the optimal value of N (sample size) for the analysis in question lies somewhere between too little and too much, akin to the essence of the Central Limit Theorem."

**Notebook Structure**

There are two distinct modules: one tailored for Intraday short-term traders and another for long-term traders. The primary divergence from a code perspective lies in the method of data retrieval, while the underlying concepts remain consistent. Please refer to the provided document dedicated to the Game of Curiosity, accessible on our Infi-flux platform under the guide section for further insights.

The comprehensive code is appended at the conclusion of each respective module.

**No need to go Super Saiyan on each cell.**

#**Module 1**

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import math

In [1]:
# Getting Data
def get_data(ticker_symbol, inter_val, prev_day):

    start_date = (datetime.now() - timedelta(days = 6)).strftime('%Y-%m-%d')
    end_date = datetime.now().strftime('%Y-%m-%d')

    cond = datetime.now() - timedelta(days = prev_day)

    df = pd.DataFrame()
    while datetime.strptime(start_date, '%Y-%m-%d') > cond :
        df = pd.concat([
            df,
            yf.download(ticker_symbol, start=start_date, end=end_date, interval = inter_val)
        ])
        end_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = 6)).strftime('%Y-%m-%d')
        start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = 6)).strftime('%Y-%m-%d')
    df.reset_index(inplace = True)
    return df
# interval values - > 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo.
# constraints applicable provided by yfinance, please check their documnetation for full transperancy

You can provide your own specific data if Yfinance does not provides the data ,
just create a Data-Frame in pandas

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
# takes DataFrame from the above function,
# the window that denotes how much a trader is willing to stay per trade which is constant for each trade,
# and condition for omitting the start and end data of each day for that instrument just to calculate all possible
# trades
# that can take place for each day. (Trades -> dummy trades)

# as the name suggest , calculating all the possible trades
def get_all_trades(window, df, intra_start, intra_end):
    targets_dict = {
        'buy' : {},
        'sell': {}
    }
    li = list(df.Datetime.apply(lambda x: str(x).split(' ')[0]).unique())
    df['dtstr'] = df.Datetime.apply(lambda x: str(x))
    for i in li:
        print('All trades for -> ',i)
        targets_dict['buy'] [i] = []
        targets_dict['sell'][i] = []
        fd = df[df.dtstr.str.contains(i)].copy()
        for ind, row in fd.loc[fd.iloc[0].name + intra_start:].iterrows():
            window_cnt = 0
            st_close = row.Close
            if ind + window - 1 - intra_end == fd.iloc[-1].name:
                break
            for indd, roww in fd.loc[ ind+1 : ].iterrows():
                window_cnt += 1
                if window_cnt == window - 1:
                    targets_dict['buy'] [i].append([str(row.Datetime), str(roww.Datetime),roww.Close - row.Close])
                    targets_dict['sell'][i].append([str(row.Datetime), str(roww.Datetime),row.Close - roww.Close])
                    break
    return targets_dict

structure of the target_dict

**targets_dict** - > ['buy', 'sell'] -> ['Dates'] -> [list of list]~columns['Enter', 'Exit','target']

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////


In [ ]:
# Selecting top trades by taking in trade_no denotes how many trades a trader places in a day which is considered as a constant for each day
# and the dictionary consisting of all day outputs from above provided each trade is Non Overlapping for that particular day,
# clearly explained in documents, Please refer the guide for this game of curiosity.

# selecting top n trades
def get_top_n_trades(trade_no, targets_dict):
    target_dict = {}
    for i in targets_dict.keys():
        target_dict[i] = {}
        for j in targets_dict[i]:
            print('Top N trades for ->', j)
            trade_cnt = trade_no
            fd = pd.DataFrame(targets_dict[i][j], columns= ['entry', 'exit', 'target']).copy()
            fd.sort_values(by = 'target' , ascending=False, inplace = True)
            target_dict[i][j] = []
            while trade_cnt > 0:
                if len(fd) == 0:
                    break
                trade = fd.iloc[0]
                try:
                    rm_li = list(
                        range(
                            fd[fd.entry == trade.entry].iloc[0].name, fd[fd.entry == trade.exit].iloc[0].name + 1
                        )
                    )
                except:
                    fd.drop([fd[fd.entry == trade.entry].iloc[0].name],inplace = True)
                    continue
                target_dict[i][j].append([trade.entry, trade.exit,trade.target])
                trade_cnt -= 1

                for l in rm_li:
                    try:
                        fd.drop([l],inplace = True)
                    except:
                        continue
    return target_dict


structure of the above get_top_n_trades() target_dic

**target_dict** - > ['buy', 'sell'] -> ['Dates'] -> [list of list]~columns['Enter', 'Exit','target']





/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////


In [ ]:
# Aggregating the trades. Trades are ranked from [ 0 to n-1] if trade_no supplied is n descending order in terms of the moment of price difference entering
# and exiting the trade for the respective sentiment of Buy and sell which corresponds to long and short respectively.

# applying CLT
def get_avg_top_n_trades(target_dict,trade_no):
    target_avg_dict = {
    }
    for i in target_dict:
        target_avg_dict[i] = {}
        for j in range(trade_no):
            target_avg_dict[i][j] = 0
        for j in target_dict[i]:
            for k in range(trade_no):
                try:
                    target_avg_dict[i][k] += target_dict[i][j][k][2]
                except:
                    #there are less no of trades for this particular day
                    continue

        tt = []
        for m in target_dict[i]:
            tt.append(len(target_dict[i][m]))
        tt = pd.DataFrame(tt,columns = ['count'])
        for j in target_avg_dict[i]:
            try:
                target_avg_dict[i][j] /= len(tt[tt['count'] > j ])
            except:
                #no trades recorded, so zero error
                continue
    return target_avg_dict


/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////


In [ ]:
# output - >
# aggregators done but by days:
# dictionary['buy'].keys() ->
# dict_keys([
# 'Tuesday',
# 'Wednesday',
# 'Thursday',
# 'Friday',
# 'Monday'
# ])

# applying CLT for/according to Weekdays
def get_avg_top_n_trades_weekdays(target_dict):
    target_avg_week_dict = {}
    deno = {}
    for i in target_dict:
        target_avg_week_dict[i] = {}
        li = {}
        for j in target_dict[i]:
            day = datetime.strptime( j, '%Y-%m-%d').strftime('%A')
            if day not in li.keys():
                li[day] = [j]
            else:
                li[day].append(j)

        # we have days
        # initialize
        for j in li:
            target_avg_week_dict[i][j] = {}
            for k in range(trade_no):
                target_avg_week_dict[i][j][k] = 0

        # adder with respect to week
        for j in li:
            for k in li[j]:
                for l in range(len(target_dict[i][k])):
                    target_avg_week_dict[i][j][l] += target_dict[i][k][l][2]

        # we need to get the denominator, bit complex.
        # need to check every day trade were there any day less than trade_no
        deno[i] = {}
        for j in li:
            deno[i][j] = []
            for k in li[j]:
                deno[i][j].append(len(target_dict[i][k]))
        # time to get the mean.
        for j in target_avg_week_dict[i]:
            for k in target_avg_week_dict[i][j]:
                try:
                    target_avg_week_dict[i][j][k] /= len([l for l in deno[i][j] if l > k])
                except:
                    # there are no duration left to trade as trade_no are just to high to fit.
                    # divide by zero error to a zero.
                    continue
        # we have it finally
    return target_avg_week_dict

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////

/////////////////////////////////////////////////////////////////////////////////////////


**The Whole Code in Action**

In [2]:

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import math
# Define the stock ticker symbol and date range
# yfinace provides data for 60 days and 7 days in one request,
# so we are iterating to get latest last month of data for particular instrunment for 2 min interval.
# listed on yfinance index.



def get_data(ticker_symbol, inter_val, prev_day):

    start_date = (datetime.now() - timedelta(days = 6)).strftime('%Y-%m-%d')
    end_date = datetime.now().strftime('%Y-%m-%d')

    cond = datetime.now() - timedelta(days = prev_day)

    df = pd.DataFrame()
    while datetime.strptime(start_date, '%Y-%m-%d') > cond :
        df = pd.concat([
            df,
            yf.download(ticker_symbol, start=start_date, end=end_date, interval = inter_val)
        ])
        end_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = 6)).strftime('%Y-%m-%d')
        start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = 6)).strftime('%Y-%m-%d')
    df.reset_index(inplace = True)
    return df
# interval values - > 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo.
# constraints applicable provided by yfinance, please check their documnetation for full transperancy

def get_all_trades(window, df, intra_start, intra_end):
    targets_dict = {
        'buy' : {},
        'sell': {}
    }
    li = list(df.Datetime.apply(lambda x: str(x).split(' ')[0]).unique())
    df['dtstr'] = df.Datetime.apply(lambda x: str(x))
    for i in li:
        print('All trades for -> ',i)
        targets_dict['buy'] [i] = []
        targets_dict['sell'][i] = []
        fd = df[df.dtstr.str.contains(i)].copy()
        for ind, row in fd.loc[fd.iloc[0].name + intra_start:].iterrows():
            window_cnt = 0
            st_close = row.Close
            if ind + window - 1 - intra_end == fd.iloc[-1].name:
                break
            for indd, roww in fd.loc[ ind+1 : ].iterrows():
                window_cnt += 1
                if window_cnt == window - 1:
                    targets_dict['buy'] [i].append([str(row.Datetime), str(roww.Datetime),roww.Close - row.Close])
                    targets_dict['sell'][i].append([str(row.Datetime), str(roww.Datetime),row.Close - roww.Close])
                    break
    return targets_dict

def get_top_n_trades(trade_no, targets_dict):
    target_dict = {}
    for i in targets_dict.keys():
        target_dict[i] = {}
        for j in targets_dict[i]:
            print('Top N trades for ->', j)
            trade_cnt = trade_no
            fd = pd.DataFrame(targets_dict[i][j], columns= ['entry', 'exit', 'target']).copy()
            fd.sort_values(by = 'target' , ascending=False, inplace = True)
            target_dict[i][j] = []
            while trade_cnt > 0:
                if len(fd) == 0:
                    break
                trade = fd.iloc[0]
                try:
                    rm_li = list(
                        range(
                            fd[fd.entry == trade.entry].iloc[0].name, fd[fd.entry == trade.exit].iloc[0].name + 1
                        )
                    )
                except:
                    fd.drop([fd[fd.entry == trade.entry].iloc[0].name],inplace = True)
                    continue
                target_dict[i][j].append([trade.entry, trade.exit,trade.target])
                trade_cnt -= 1

                for l in rm_li:
                    try:
                        fd.drop([l],inplace = True)
                    except:
                        continue
    return target_dict



def get_avg_top_n_trades(target_dict,trade_no):
    target_avg_dict = {
    }
    for i in target_dict:
        target_avg_dict[i] = {}
        for j in range(trade_no):
            target_avg_dict[i][j] = 0
        for j in target_dict[i]:
            for k in range(trade_no):
                try:
                    target_avg_dict[i][k] += target_dict[i][j][k][2]
                except:
                    #there are less no of trades for this particular day
                    continue

        tt = []
        for m in target_dict[i]:
            tt.append(len(target_dict[i][m]))
        tt = pd.DataFrame(tt,columns = ['count'])
        for j in target_avg_dict[i]:
            try:
                target_avg_dict[i][j] /= len(tt[tt['count'] > j ])
            except:
                #no trades recorded, so zero error
                continue
    return target_avg_dict


def get_avg_top_n_trades_weekdays(target_dict):
    target_avg_week_dict = {}
    deno = {}
    for i in target_dict:
        target_avg_week_dict[i] = {}
        li = {}
        for j in target_dict[i]:
            day = datetime.strptime( j, '%Y-%m-%d').strftime('%A')
            if day not in li.keys():
                li[day] = [j]
            else:
                li[day].append(j)

        # we have days
        # initialize
        for j in li:
            target_avg_week_dict[i][j] = {}
            for k in range(trade_no):
                target_avg_week_dict[i][j][k] = 0

        # adder with respect to week
        for j in li:
            for k in li[j]:
                for l in range(len(target_dict[i][k])):
                    target_avg_week_dict[i][j][l] += target_dict[i][k][l][2]

        # we need to get the denominator, bit complex.
        #need to check every day trade were there any day less than trade_no
        deno[i] = {}
        for j in li:
            deno[i][j] = []
            for k in li[j]:
                deno[i][j].append(len(target_dict[i][k]))
        # time to get the mean.
        for j in target_avg_week_dict[i]:
            for k in target_avg_week_dict[i][j]:
                try:
                    target_avg_week_dict[i][j][k] /= len([l for l in deno[i][j] if l > k])
                except:
                    # there are no duration left to trade as trade_no are just to high to fit.
                    #divide by zero error to a zero.
                    continue
        # we have it finally
    return target_avg_week_dict

data_days = 30       # retriving data from Yfinance for 30 days
window = 25          # 25 minutes duration = window * interval
trade_no = 8         # per day maximum trade taken
intra_start = 20     # Ignoring 20 minutes = (interval_start * interval) the start of the day
intra_end = 20       # Ignoring 20 minutes = (interval_start * interval) the end of the day
interval = '1m'      # timeframe of the OHLCV data
ticker = 'TSLA'      # Desired instrument listed in financial market managed by Yfinance


# df = get_data(ticker, interval, data_days).sort_values(by = 'Datetime').reset_index(drop = True)
if interval in ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h']:
    df = get_data(
        ticker,
        interval,
        data_days
    ).sort_values(
        by = 'Datetime'
    ).reset_index(
        drop = True
    )
else:
    print('invalid Interval - please re-enter')
targets_dict = get_all_trades(window, df, intra_start, intra_end)
target_dict = get_top_n_trades(trade_no,targets_dict)
target_avg_dict = get_avg_top_n_trades(target_dict,trade_no)
target_avg_week_dict = get_avg_top_n_trades_weekdays(target_dict)
print(target_avg_dict)
print(target_avg_week_dict)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


All trades for ->  2024-03-05
All trades for ->  2024-03-06
All trades for ->  2024-03-07
All trades for ->  2024-03-08
All trades for ->  2024-03-11
All trades for ->  2024-03-12
All trades for ->  2024-03-13
All trades for ->  2024-03-14
All trades for ->  2024-03-15
All trades for ->  2024-03-18
All trades for ->  2024-03-19
All trades for ->  2024-03-20
All trades for ->  2024-03-21
All trades for ->  2024-03-22
All trades for ->  2024-03-25
All trades for ->  2024-03-26
All trades for ->  2024-03-27
All trades for ->  2024-03-28
Top N trades for -> 2024-03-05
Top N trades for -> 2024-03-06
Top N trades for -> 2024-03-07
Top N trades for -> 2024-03-08
Top N trades for -> 2024-03-11
Top N trades for -> 2024-03-12
Top N trades for -> 2024-03-13
Top N trades for -> 2024-03-14
Top N trades for -> 2024-03-15
Top N trades for -> 2024-03-18
Top N trades for -> 2024-03-19
Top N trades for -> 2024-03-20
Top N trades for -> 2024-03-21
Top N trades for -> 2024-03-22
Top N trades for -> 2024-0

In [7]:
print('Configuration of the investment strategy')
print('Number of data available by Yfinance , the N sample Size',data_days)
print('Duration          : ',window)
print('Trade_no each day : ',trade_no)
print('Intra_start       : ',intra_start)
print('Intra_end         : ',intra_end)
print('Interval          : ',interval)
print('Ticker            : ',ticker)



Configuration of the investment strategy
Number of data available by Yfinance , the N sample Size 30
Duration          :  25
Trade_no each day :  8
Intra_start       :  20
Intra_end         :  20
Interval          :  1m
Ticker            :  TSLA


////////////////////////////////////////////////////////

In [5]:
print('target_avg_dict keys : ', target_avg_dict.keys())
print('buy keys            : ',target_avg_dict['buy'].keys())
print('sell keys           : ',target_avg_dict['sell'].keys())

target_avg_dict keys :  dict_keys(['buy', 'sell'])
buy keys            :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7])
sell keys           :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7])


In [8]:
# Normal Output
pd.DataFrame(
    [
        target_avg_dict['buy']
        ]
    ) # Columns are rank in descending payouts on one share invested with buy sentiment

,0,1,2,3,4,5,6,7
0,2.108701,1.432866,1.137645,0.87478,0.667387,0.430845,0.149272,-0.062292


In [9]:
pd.DataFrame(
    [
        target_avg_dict['sell']
        ]
    )# Columns are rank in descending payouts on one share invested with sell sentiment

,0,1,2,3,4,5,6,7
0,2.050941,1.513432,1.12343,0.926,0.745195,0.47907,0.150795,-0.003967


The above output denotes the expected return of maximum payoff that could be achieved if one share is invested for the above configuration provided for investment strategy for respective sentiment ['Buy','Sell']

////////////////////////////////////////////////////////////////////////

In [6]:
print('target_avg_week_dict keys : ' , target_avg_week_dict.keys())
print('buy keys : ' , target_avg_week_dict['buy'].keys())
print('sell keys : ' , target_avg_week_dict['sell'].keys())
print('buy Monday keys  : ' , target_avg_week_dict['buy']['Monday'].keys())
print('sell Monday keys : ' , target_avg_week_dict['sell']['Monday'].keys())

target_avg_week_dict keys :  dict_keys(['buy', 'sell'])
buy keys :  dict_keys(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday'])
sell keys :  dict_keys(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Monday'])
buy Monday keys  :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7])
sell Monday keys :  dict_keys([0, 1, 2, 3, 4, 5, 6, 7])


In [13]:
# for Week_days
pd.DataFrame(
    [
        target_avg_week_dict['buy']['Wednesday']   # change the variable
        ]
    ) # replace with desired day of the week to know the possible payout
    #~ Monday, Tuesday, wednesday, Thrusday, Friday, Sunday

,0,1,2,3,4,5,6,7
0,2.263458,1.526424,1.247078,0.94582,0.677399,0.492176,0.129498,-0.015569


In [12]:
pd.DataFrame([
    target_avg_week_dict['sell']['Wednesday'] # desired day of the week
    ])# replace with desired day of the week to know the possible payout
    #~ Monday, Tuesday, wednesday, Thrusday, Friday, Sunday

,0,1,2,3,4,5,6,7
0,2.077103,1.794254,1.12252,0.913025,0.689377,0.418999,0.102428,-0.216797


The above output is for respective sentiment, but for each weekday, you can check by replacing the variable Monday in the above code with dates you are interested in.

#**Module 2**
The Whole Code for Long term investing strategies

In [14]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import math
# for long term trades it is easy
def get_all_trades_lng(window, df):
# window shoud be atleast less then len(data)
    targets_dict = {
        'buy' : [],
        'sell': []
    }
    dtcond = 1 if 'Datetime' in df.columns else 0
    for ind, row in df.iterrows():
        window_cnt = 0
        st_close = row.Close
        if ind + window - 1  == df.iloc[-1].name:
            break
        for indd, roww in df.loc[ ind+1 : ].iterrows():
            window_cnt += 1
            if window_cnt == window - 1:
                if dtcond == 1:
                    targets_dict['buy'] .append([str(row.Datetime), str(roww.Datetime),roww.Close - row.Close])
                    targets_dict['sell'].append([str(row.Datetime), str(roww.Datetime),row.Close - roww.Close])
                else:
                    targets_dict['buy'] .append([str(row.Date), str(roww.Date),roww.Close - row.Close])
                    targets_dict['sell'].append([str(row.Date), str(roww.Date),row.Close - roww.Close])

                break

    return targets_dict


def get_data_lng(ticker_symbol, inter_val, prev_day, looper):

    start_date = (datetime.now() - timedelta(days = looper)).strftime('%Y-%m-%d')
    end_date = datetime.now().strftime('%Y-%m-%d')

    cond = datetime.now() - timedelta(days = prev_day)

    df = pd.DataFrame()
    while datetime.strptime(start_date, '%Y-%m-%d') > cond :
        df = pd.concat([
            df,
            yf.download(ticker_symbol, start=start_date, end=end_date, interval = inter_val)
        ])
        end_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = looper)).strftime('%Y-%m-%d')
        start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days = looper)).strftime('%Y-%m-%d')
    df.reset_index(inplace = True)
    return df
# interval values - > 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo.

def get_top_n_trades_lng(trade_no, targets_dict):
    target_dict = {}
    for i in targets_dict.keys():
        target_dict[i] = {}
        fd = pd.DataFrame(targets_dict[i], columns= ['entry', 'exit', 'target']).copy()
        fd.sort_values(by = 'target' , ascending=False, inplace = True)
        trade_cnt = trade_no
        target_dict[i] = []
        while trade_cnt > 0:
            if len(fd) == 0:
                break
            trade = fd.iloc[0]
            try:
                rm_li = list(
                    range(
                        fd[fd.entry == trade.entry].iloc[0].name, fd[fd.entry == trade.exit].iloc[0].name + 1
                    )
                )
            except:
                fd.drop([fd[fd.entry == trade.entry].iloc[0].name],inplace = True)
                continue
            target_dict[i].append([trade.entry, trade.exit,trade.target])
            trade_cnt -= 1

            for l in rm_li:
                try:
                    fd.drop([l],inplace = True)
                except:
                    continue
    return target_dict


#driver parameters
window = 5
trade_no = 8
data_days = 100
interval = '1d'
ticker = 'TSLA'
looper = 20

if interval in ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h']:
    looper = 6
    df = get_data_lng(ticker, interval, data_days, looper)
    df = df.sort_values(by = 'Datetime').reset_index(drop = True)
else:
    df = get_data_lng(ticker, interval, data_days, looper)
    df = df.sort_values(by = 'Date').reset_index(drop = True)
targets_dict = get_all_trades_lng( window, df)
target_dict = get_top_n_trades_lng(trade_no, targets_dict)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [15]:
print(target_dict)

{'buy': [['2024-03-14 00:00:00', '2024-03-20 00:00:00', 13.160003662109375], ['2024-02-05 00:00:00', '2024-02-09 00:00:00', 12.510009765625], ['2024-02-12 00:00:00', '2024-02-16 00:00:00', 11.819992065429688], ['2024-02-23 00:00:00', '2024-02-29 00:00:00', 9.910003662109375], ['2024-01-26 00:00:00', '2024-02-01 00:00:00', 5.6100006103515625], ['2024-03-06 00:00:00', '2024-03-12 00:00:00', 1.0], ['2024-01-18 00:00:00', '2024-01-24 00:00:00', -4.0500030517578125]], 'sell': [['2024-01-19 00:00:00', '2024-01-25 00:00:00', 29.55999755859375], ['2024-02-29 00:00:00', '2024-03-06 00:00:00', 25.340011596679688], ['2024-01-10 00:00:00', '2024-01-17 00:00:00', 18.389999389648438], ['2024-03-11 00:00:00', '2024-03-15 00:00:00', 14.199996948242188], ['2024-01-30 00:00:00', '2024-02-05 00:00:00', 10.529998779296875], ['2024-02-16 00:00:00', '2024-02-23 00:00:00', 7.9799957275390625], ['2024-02-07 00:00:00', '2024-02-13 00:00:00', 3.55999755859375]]}


Module 2 Output

In [16]:
pd.DataFrame(
    target_dict['buy'],
    columns = ['Entry', 'Exit', 'target']
    )

,Entry,Exit,target
0,2024-03-14 00:00:00,2024-03-20 00:00:00,13.160004
1,2024-02-05 00:00:00,2024-02-09 00:00:00,12.510010
2,2024-02-12 00:00:00,2024-02-16 00:00:00,11.819992
3,2024-02-23 00:00:00,2024-02-29 00:00:00,9.910004
4,2024-01-26 00:00:00,2024-02-01 00:00:00,5.610001
5,2024-03-06 00:00:00,2024-03-12 00:00:00,1.000000
6,2024-01-18 00:00:00,2024-01-24 00:00:00,-4.050003


In [17]:
pd.DataFrame(
    target_dict['sell'],
    columns = ['Entry', 'Exit', 'target']
    )

,Entry,Exit,target
0,2024-01-19 00:00:00,2024-01-25 00:00:00,29.559998
1,2024-02-29 00:00:00,2024-03-06 00:00:00,25.340012
2,2024-01-10 00:00:00,2024-01-17 00:00:00,18.389999
3,2024-03-11 00:00:00,2024-03-15 00:00:00,14.199997
4,2024-01-30 00:00:00,2024-02-05 00:00:00,10.529999
5,2024-02-16 00:00:00,2024-02-23 00:00:00,7.979996
6,2024-02-07 00:00:00,2024-02-13 00:00:00,3.559998


**We** tried our best to make the above code as readable as possible. We wish you the best.
Detail description of each variable are given in the game of curiosity document. Please read the documents if some concepts are not clear.

Thank you

Be careful

By Infi-flux